# SE-Sync benchmarking

This notebook uses SE-Sync's Python bindings to permit easy benchmarking against a collection of large-scale SLAM benchmarks

In [4]:
import sys
import numpy as np
import time

import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from IPython.display import display  # To get pretty tabular output w/ PANDAS
import pickle

import PySESync

### Test configuration

#### Test benchmarks

In [6]:
# Location of folder containing the benchmarks
data_folder = "../data/"

# Specific list of files to process
# files = ["manhattan", "city10000", "MIT", "CSAIL", "intel", "sphere2500", "torus3D", "parking-garage", "rim"]
files = ["sphere2500"]

#### SE-Sync configurations

In [7]:
# Set list of SE-Sync configurations to process

# Common configurations
num_threads = 4
verbose = False

opts_list = [PySESync.SESyncOpts() for i in range(6)]

# Config 0: Simplified w/ chordal init
opts_list[0].formulation = PySESync.Formulation.Simplified
opts_list[0].initialization = PySESync.Initialization.Chordal
opts_list[0].num_threads = 4
opts_list[0].verbose = verbose

# Config 1: Simplified w/ random init
opts_list[1].formulation = PySESync.Formulation.Simplified
opts_list[1].initialization = PySESync.Initialization.Random
opts_list[1].num_threads = 4
opts_list[1].verbose = verbose

# Config 2: Explicit w/ chordal init
opts_list[2].formulation = PySESync.Formulation.Explicit
opts_list[2].initialization = PySESync.Initialization.Chordal
opts_list[2].num_threads = 4
opts_list[2].verbose = verbose

# Config 3: Explicit w/ random init
opts_list[3].formulation = PySESync.Formulation.Explicit
opts_list[3].initialization = PySESync.Initialization.Random
opts_list[3].num_threads = 4
opts_list[3].verbose = verbose

# Config 4: SOSync w/ chordal init
opts_list[4].formulation = PySESync.Formulation.SOSync
opts_list[4].initialization = PySESync.Initialization.Chordal
opts_list[4].num_threads = 4
opts_list[4].verbose = verbose

# Config 5: SOSync w/ random init
opts_list[5].formulation = PySESync.Formulation.SOSync
opts_list[5].initialization = PySESync.Initialization.Random
opts_list[5].num_threads = 4
opts_list[5].verbose = verbose


### Run tests!

In [8]:
# Iterate over the list of benchmarks

data = []

start_time = time.time()
for k, f in enumerate(files):
    
    # Construct complete path to this file
    filename = data_folder + f + ".g2o"

    # Read measurements out of this file
    measurements, num_poses = PySESync.read_g2o_file(filename)
    
    # Get dimension of problem
    d = measurements[0].R.shape[0]
    
    # For each configuration in the test set 
    for c, opts in enumerate(opts_list):
        
        # Set initial relaxation rank to be the dimension of this test problem
        opts.r0 = d
        
        # Run SE-Sync with this configuration!
        result = PySESync.SESync(measurements, opts)
        
        # Record results for this test
        data.append({"Config": c, "Dataset" : f, "Status" : result.status.name,  \
                    "Fxhat" : result.Fxhat, "GradNorm" : result.gradnorm, \
                    "SubOptBound" : result.suboptimality_bound, \
                    "TotalTime" : result.total_computation_time, \
                    #"InitTime" : result.initialization_time, \
                    "OptTime" : sum(l[-1] for l in result.elapsed_optimization_times), \
                    "OptIters" : sum(len(l) for l in result.elapsed_optimization_times), \
                    "HessVecProds" : sum(map(sum, result.Hessian_vector_products)), \
                    "VerTime" : sum(result.verification_times), \
                    "VerIters" : sum(result.LOBPCG_iters), \
                    "FinalRank" : result.Yopt.shape[0]})
        

end_time = time.time()

elapsed_time = (end_time - start_time)

print("All tests finished.  Total computation time: %g seconds" % elapsed_time)

All tests finished.  Total computation time: 42.2859 seconds


### Analysis

Create a pandas datafrom from the recorded results for further processing

In [9]:
df = pd.DataFrame(data)
display(df)

,Config,Dataset,Status,Fxhat,GradNorm,SubOptBound,TotalTime,OptTime,OptIters,HessVecProds,VerTime,VerIters,FinalRank
0,0,sphere2500,GlobalOpt,1687.005678,0.084318,4.001777e-11,0.426,0.115,7,21,0.026,0,3
1,1,sphere2500,GlobalOpt,1687.005672,0.025772,-2.346193e-06,3.505,3.417,76,721,0.043,1,4
2,2,sphere2500,GlobalOpt,1687.005671,0.013150,-1.989085e-02,0.606,0.283,8,57,0.026,0,3
3,3,sphere2500,GlobalOpt,1687.005674,0.055862,-1.262584e-01,35.755,35.700,264,8215,0.036,1,4
4,4,sphere2500,GlobalOpt,885.362577,0.000202,-1.023182e-12,0.319,0.020,4,3,0.016,0,3
5,5,sphere2500,GlobalOpt,885.362577,0.000455,-4.584990e-10,0.406,0.371,52,37,0.021,1,4


Save this data to disk

In [12]:
df.to_pickle("/home/jason/DPGO/SE-Sync/python/SESync_benchmarking.pkl")

Read data from disk

In [13]:
df = pd.read_pickle("/home/jason/DPGO/SE-Sync/python/SESync_benchmarking.pkl")